In [1]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import boto3

In [2]:
s3 = boto3.client('s3')

In [3]:
#Daños 
yesterday = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
bucket_name = 'hdi-sagemaker-project-co'
s3_path_A = f"data-science/nppm-latam/fraude/daily_claims_colombia/daily_claims_colombia_danios/siniestros_novedades_diario_danios_{yesterday}.xlsx"
local_file_name_A = 'file_A.xlsx'

s3.download_file(bucket_name, s3_path_A, local_file_name_A)
file_A = pd.read_excel(local_file_name_A)

In [4]:
#Hurto
s3_path_B = f"data-science/nppm-latam/fraude/daily_claims_colombia/daily_claims_colombia_hurto/siniestros_novedades_diario_hurto_{yesterday}.xlsx"
local_file_name_B = 'file_B.xlsx'

s3.download_file(bucket_name, s3_path_B, local_file_name_B)
file_B = pd.read_excel(local_file_name_B)

In [5]:
columns_to_keep = ['numero_siniestro', 'placa_vehiculo', 'descripcion_riesgo', 'tipo_iden_aseg', 'nro_iden_aseg', 'servicio']

In [6]:
file_A = file_A[columns_to_keep]
file_B = file_B[columns_to_keep]

In [7]:
file_A.head(2)

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular
1,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular


In [8]:
print(file_A.shape[0]),print(file_B.shape[0]) 

53305
2638


(None, None)

In [9]:
# Anexar las filas de file_A y file_B
file_E = pd.concat([file_A, file_B])

In [10]:
file_E.head(2)

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular
1,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular


In [11]:
print(file_E.shape[0])

55943


In [12]:
# Eliminar duplicados 
file_E = file_E.drop_duplicates()
file_E.reset_index(drop = True, inplace = True)
file_E.head(2)

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular


In [13]:
print(file_E.shape[0])

124


In [14]:
# Cargar resumen de tiempos 
file_C = pd.read_excel("ResumenTiemposEspecial  122024.xlsx") #Resumen de tipos sharepoint
file_C.to_excel("file_C.xlsx", index = False)


In [15]:
# columnas que se utilizaran y renombrar 
columns_to_keep_C = ['Clave', 'Amparo', 'Siniestro', 'Placa', 'FechaSin', 'CedConductor']
rename_columns_C = {'Siniestro': 'numero_siniestro', 'Placa': 'placa_vehiculo'}

In [16]:
# filtrar y renombrar columnas
file_C = file_C[columns_to_keep_C].rename(columns=rename_columns_C)

In [17]:
file_C.head(2)

,Clave,Amparo,numero_siniestro,placa_vehiculo,FechaSin,CedConductor
0,800772,PPD,1067405,ISQ167,2024-01-01,1040043739
1,800773,PPD,1067406,JGS782,2023-12-31,1061737635


In [18]:
file_C.dtypes

Clave                        int64
Amparo                      object
numero_siniestro            object
placa_vehiculo              object
FechaSin            datetime64[ns]
CedConductor                object
dtype: object

In [19]:
file_E.dtypes

numero_siniestro       int64
placa_vehiculo        object
descripcion_riesgo    object
tipo_iden_aseg        object
nro_iden_aseg          int64
servicio              object
dtype: object

In [20]:
# Filtrar file_C para mantener solo las filas donde Amparo es 'PPD' o 'PTD' o 'PPH' o 'PTH'file_C = file_C[(file_C['Amparo'] == 'PPD') | (file_C['Amparo'] == 'PTD')
file_C = file_C[(file_C['Amparo'] == 'PPD') | (file_C['Amparo'] == 'PTD')
                | (file_C['Amparo'] == 'PPH') | (file_C['Amparo'] == 'PTH')]
file_C.reset_index(inplace = True, drop = True)

In [21]:
file_C.head(2)

,Clave,Amparo,numero_siniestro,placa_vehiculo,FechaSin,CedConductor
0,800772,PPD,1067405,ISQ167,2024-01-01,1040043739
1,800773,PPD,1067406,JGS782,2023-12-31,1061737635


In [22]:
# Filtrar filas con solo números en numero_siniestro
file_C = file_C[file_C['numero_siniestro'].apply(lambda x: x.isdigit())]

# Convertir la columna 'numero_siniestro' en numérica
file_C['numero_siniestro'] = file_C['numero_siniestro'].astype(int)

In [23]:
file_C.dtypes

Clave                        int64
Amparo                      object
numero_siniestro             int64
placa_vehiculo              object
FechaSin            datetime64[ns]
CedConductor                object
dtype: object

In [24]:
# Para optimizar el vlookup, primero crearemos un diccionario a partir de df_file_C_filtered para una búsqueda más rápida
lookup_dict = file_C.set_index('numero_siniestro')['CedConductor'].to_dict()

In [25]:
cedula_conductor = []
for index, value in file_C['numero_siniestro'].items():
    ced_conductor = lookup_dict.get(value)
    cedula_conductor.append(ced_conductor)

cedula_conductor[-3:]

['1003404790', '1144036107', '1107514145']

In [26]:
file_E.head(2)

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular


In [27]:
file_E['numero_siniestro'].dtype

dtype('int64')

In [28]:
file_C.head(2)

,Clave,Amparo,numero_siniestro,placa_vehiculo,FechaSin,CedConductor
0,800772,PPD,1067405,ISQ167,2024-01-01,1040043739
1,800773,PPD,1067406,JGS782,2023-12-31,1061737635


In [29]:
file_C['numero_siniestro'].dtype

dtype('int64')

In [30]:
file_C['FechaSin'].dtype

dtype('<M8[ns]')

In [31]:
file_C['FechaSin'] = pd.to_datetime(file_C['FechaSin'])

In [32]:
#match = file_C[(file_C['numero_siniestro'] == 0) & (file_C['placa_vehiculo'] == 'HPX193')]
#fecha_sin = match.iloc[0]['FechaSin']
#fecha_sin

In [33]:
# Función para realizar la operación vlookup-like
def vlookup(row):
    # Búsqueda por numero_siniestro
    numero_siniestro = row['numero_siniestro']
    ced_conductor = lookup_dict.get(numero_siniestro)

    if pd.isna(ced_conductor) and numero_siniestro == 0:
        # Si numero_siniestro es 0, intente vlookup por placa_vehiculo
        placa_vehiculo = row['placa_vehiculo']
        match = file_C[(file_C['numero_siniestro'] == 0) & (file_C['placa_vehiculo'] == placa_vehiculo)]
        if not match.empty:
            # Revisar si FechaSin está dentro del último mes
            fecha_sin = match.iloc[0]['FechaSin']
            current_date = datetime.today()
            if isinstance(fecha_sin, datetime) and fecha_sin > (current_date - timedelta(days=30)):
                ced_conductor = match.iloc[0]['CedConductor']

    return ced_conductor


# Aplicar la función optimizada vlookup a file_E
file_E['nro_iden_conduc'] = file_E.apply(vlookup, axis=1)

# Comprobar si nro_iden_conduc coincide con nro_iden_aseg
file_E['igual'] = file_E['nro_iden_conduc'] == file_E['nro_iden_aseg']

# Visualización de las primeras filas para la verificación
file_E.head(2)

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio,nro_iden_conduc,igual
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular,None,False
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular,None,False


In [34]:
file_E["igual"] = file_E["igual"].astype(int)

In [35]:
file_E['nro_iden_conduc'].isnull().sum()

np.int64(122)

In [36]:
# Ahora, vamos a replicar el último paso pero usando el archivo D y en los casos en los que aún faltaba nro_iden_conduc en el archivo E

In [37]:
# Cargar file_D 

df_file_D = pd.read_csv("claims_sipo_adp_2025-01-12.csv") #SIPO email
df_file_D.to_csv('file_D.csv', index = False)

# Mantenga solo las columnas específicas y cámbieles el nombre en file_D
columns_to_keep_D = ['clave', 'amparo', 'siniestro', 'placa', 'cedconductor']
rename_columns_D = {'siniestro': 'numero_siniestro', 'placa': 'placa_vehiculo'}
file_D = df_file_D[columns_to_keep_D].rename(columns=rename_columns_D)

# Convertir 'numero_siniestro' a numérico
file_D['numero_siniestro'] = pd.to_numeric(file_D['numero_siniestro'], errors='coerce')
file_D['cedconductor'] = pd.to_numeric(file_D['cedconductor'], errors='coerce')

# Crear un diccionario para lookup de file_D
lookup_dict_D = file_D.set_index('numero_siniestro')['cedconductor'].to_dict()

# Función para realizar vlookup para los valores faltantes en file_E usando file_D
def vlookup_missing_values(row):
    if pd.isna(row['nro_iden_conduc']):
        # Primer intento de coincidencia por numero_siniestro de file_D
        numero_siniestro = row['numero_siniestro']
        ced_conductor = lookup_dict_D.get(numero_siniestro)

        if pd.isna(ced_conductor) and numero_siniestro == 0:
            # Si numero_siniestro es 0, intente vlookup por placa_vehiculo
            placa_vehiculo = row['placa_vehiculo']
            match = file_D[(file_D['numero_siniestro'] == 0) & 
                                       (file_D['placa_vehiculo'] == placa_vehiculo)]
            if not match.empty:
                ced_conductor = match.iloc[0]['cedconductor']

        return ced_conductor
    else:
        return row['nro_iden_conduc']

# Aplicar la función a file_E
file_E['nro_iden_conduc'] = file_E.apply(vlookup_missing_values, axis=1)

# Visualización de las primeras filas para la verificación
file_E.head()

/tmp/ipykernel_25480/2521107428.py:3: DtypeWarning: Columns (48,80,89,90,95,106,108,118,136,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df_file_D = pd.read_csv("claims_sipo_adp_2025-01-12.csv") #SIPO email


,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio,nro_iden_conduc,igual
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular,19266175.0,0
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular,None,0
2,1126594,NJP712,NJP712 - VOLKSWAGEN - NIVUS - COMFORTLINE TP 1...,Cédula Ciudadanía,79304704,Particular,79304704.0,0
3,1126588,LLO507,LLO507 - KIA - PICANTO [3] - VIBRANTMT 1250C,Cédula Ciudadanía,32711460,Particular,32711460.0,0
4,1126589,IPX437,IPX437 - KIA - CERATO GEO - 1.6 MT 1600CC AB TC,Cédula Ciudadanía,66827858,Particular,66827858.0,0


In [38]:
file_E.dtypes

numero_siniestro       int64
placa_vehiculo        object
descripcion_riesgo    object
tipo_iden_aseg        object
nro_iden_aseg          int64
servicio              object
nro_iden_conduc       object
igual                  int64
dtype: object

In [39]:
# Función para ajustar la longitud de los números en nro_iden_aseg y nro_iden_conduc para tipo_iden_aseg = 'NIT'
def adjust_number_length(row):
    if row['tipo_iden_aseg'] == 'NIT':
        # Ajustar nro_iden_aseg
        if len(str(row['nro_iden_aseg'])) > 9:
            row['nro_iden_aseg'] = str(row['nro_iden_aseg'])[:9]

            # Si nro_iden_conduc era el mismo que nro_iden_aseg antes de la manteca, ajústelo también
            if row['nro_iden_conduc'] == row['nro_iden_aseg']:
                row['nro_iden_conduc'] = row['nro_iden_aseg']

    return row

# Aplicvar la función a file_E
file_E = file_E.apply(adjust_number_length, axis=1)

# Visualización de las primeras filas para la verificación
file_E.head(2)

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio,nro_iden_conduc,igual
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular,19266175.0,0
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular,None,0


In [40]:
file_E.to_excel("filef.xlsx")

In [41]:
file_E.dtypes

numero_siniestro       int64
placa_vehiculo        object
descripcion_riesgo    object
tipo_iden_aseg        object
nro_iden_aseg         object
servicio              object
nro_iden_conduc       object
igual                  int64
dtype: object

In [42]:
file_E['nro_iden_aseg'] = pd.to_numeric(file_E['nro_iden_aseg'], errors='coerce')

In [43]:
file_E['nro_iden_conduc'].isnull().sum()

np.int64(4)

In [44]:
# Ajustando la longitud de 'nro_iden_conduc' para tipo_iden_aseg = 'NIT' e igual = 1
def adjust_nro_iden_conduc_length(row):
    if row['tipo_iden_aseg'] == 'NIT' and row['igual'] == 1:
        # Asegúrese de que nro_iden_conduc sea una cadena para la comprobación de longitud
        nro_iden_conduc_str = str(row['nro_iden_conduc'])

        if len(nro_iden_conduc_str) > 9:
            # Reducir a los primeros 9 números
            row['nro_iden_conduc'] = nro_iden_conduc_str[:9]

    return row

# Aplicar la función a file_E
file_E = file_E.apply(adjust_nro_iden_conduc_length, axis=1)

# Visualización de las primeras filas para la verificación
file_E.head()


,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio,nro_iden_conduc,igual
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular,19266175.0,0
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular,None,0
2,1126594,NJP712,NJP712 - VOLKSWAGEN - NIVUS - COMFORTLINE TP 1...,Cédula Ciudadanía,79304704,Particular,79304704.0,0
3,1126588,LLO507,LLO507 - KIA - PICANTO [3] - VIBRANTMT 1250C,Cédula Ciudadanía,32711460,Particular,32711460.0,0
4,1126589,IPX437,IPX437 - KIA - CERATO GEO - 1.6 MT 1600CC AB TC,Cédula Ciudadanía,66827858,Particular,66827858.0,0


In [45]:
file_E.dtypes

numero_siniestro       int64
placa_vehiculo        object
descripcion_riesgo    object
tipo_iden_aseg        object
nro_iden_aseg          int64
servicio              object
nro_iden_conduc       object
igual                  int64
dtype: object

In [46]:
file_E['nro_iden_conduc'] = pd.to_numeric(file_E['nro_iden_conduc'], errors='coerce')

In [47]:
file_E.dtypes

numero_siniestro        int64
placa_vehiculo         object
descripcion_riesgo     object
tipo_iden_aseg         object
nro_iden_aseg           int64
servicio               object
nro_iden_conduc       float64
igual                   int64
dtype: object

In [48]:
# Eliminación de la columna 'igual'
file_E = file_E.drop(columns=['igual'])

# Recreando la columna 'igual' para comprobar si nro_iden_conduc coincide con nro_iden_aseg
file_E['igual'] = file_E['nro_iden_conduc'] == file_E['nro_iden_aseg']
file_E["igual"] = file_E["igual"].astype(int)
# Visualización de las primeras filas para la verificación
file_E.head()

,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio,nro_iden_conduc,igual
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular,19266175.0,1
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular,NaN,0
2,1126594,NJP712,NJP712 - VOLKSWAGEN - NIVUS - COMFORTLINE TP 1...,Cédula Ciudadanía,79304704,Particular,79304704.0,1
3,1126588,LLO507,LLO507 - KIA - PICANTO [3] - VIBRANTMT 1250C,Cédula Ciudadanía,32711460,Particular,32711460.0,1
4,1126589,IPX437,IPX437 - KIA - CERATO GEO - 1.6 MT 1600CC AB TC,Cédula Ciudadanía,66827858,Particular,66827858.0,1


In [49]:
file_E["igual"] = file_E["igual"].astype(int)

In [50]:
file_E.dtypes

numero_siniestro        int64
placa_vehiculo         object
descripcion_riesgo     object
tipo_iden_aseg         object
nro_iden_aseg           int64
servicio               object
nro_iden_conduc       float64
igual                   int64
dtype: object

In [51]:
# Asigne 'igual' igual a 1 para los casos en los que 'nro_iden_conduc' sea un valor faltante
file_E['igual'] = file_E['igual'] | file_E['nro_iden_conduc'].isna()

# Visualización de las primeras filas para la verificación
file_E.head()


,numero_siniestro,placa_vehiculo,descripcion_riesgo,tipo_iden_aseg,nro_iden_aseg,servicio,nro_iden_conduc,igual
0,1126553,KOX390,KOX390 - JEEP - RENEGADE - SPORT MT 1800CC 4X2...,Cédula Ciudadanía,19266175,Particular,19266175.0,True
1,1126683,EPM424,EPM424 - MAZDA - CX5 [2] - TOURINGTP 2500C,Cédula Ciudadanía,1088260313,Particular,NaN,True
2,1126594,NJP712,NJP712 - VOLKSWAGEN - NIVUS - COMFORTLINE TP 1...,Cédula Ciudadanía,79304704,Particular,79304704.0,True
3,1126588,LLO507,LLO507 - KIA - PICANTO [3] - VIBRANTMT 1250C,Cédula Ciudadanía,32711460,Particular,32711460.0,True
4,1126589,IPX437,IPX437 - KIA - CERATO GEO - 1.6 MT 1600CC AB TC,Cédula Ciudadanía,66827858,Particular,66827858.0,True


In [52]:
file_E["igual"] = file_E["igual"].astype(int)

In [53]:
file_E.dtypes

numero_siniestro        int64
placa_vehiculo         object
descripcion_riesgo     object
tipo_iden_aseg         object
nro_iden_aseg           int64
servicio               object
nro_iden_conduc       float64
igual                   int64
dtype: object

In [54]:
file_E = file_E[['numero_siniestro', 'placa_vehiculo', 'tipo_iden_aseg','nro_iden_aseg', 'nro_iden_conduc', 'igual', 
                 'descripcion_riesgo', 'servicio']]

In [55]:
file_E.to_excel("pruebarunt.xlsx")

In [56]:
# El dataframe con columnas a, b, c, d, e, y f
existing_dataframe = pd.read_excel("pruebarunt.xlsx")
# Cree una lista vacía para almacenar las filas del nuevo dataframe
new_data = []

# Bucle a través del dataframe existente
for index, row in existing_dataframe.iterrows():
    if row['igual'] == 1:
        # Agregue una sola fila al nuevo marco de datos con 'identificacion' como valor de la columna d
        new_data.append({'placa': row['placa_vehiculo'], 'identificacion': row['nro_iden_aseg'], 'tipo_identificacion': row['tipo_iden_aseg'], 'tipo_vehiculo': row['descripcion_riesgo']})
    else:
        # Agregue dos filas al nuevo marco de datos, una con 'identificación' como valor de la columna d
        # y otro con 'identificación' como columna e valor
        new_data.append({'placa': row['placa_vehiculo'], 'identificacion': row['nro_iden_aseg'], 'tipo_identificacion': row['tipo_iden_aseg'], 'tipo_vehiculo': row['descripcion_riesgo']})
        new_data.append({'placa': row['placa_vehiculo'], 'identificacion': row['nro_iden_conduc'], 'tipo_identificacion': row['tipo_iden_aseg'], 'tipo_vehiculo': row['descripcion_riesgo']})

# Crear el nuevo marco de datos a partir de la lista de datos
new_dataframe = pd.DataFrame(new_data)
new_dataframe.to_excel("muestra_Liberty_INTEMPO.xlsx", index=False)

In [57]:
import pandas as pd
def procesar_placas(dataframe):
    """
   Procesa un DataFrame con las siguientes reglas:
   1. Filtra únicamente los valores con 'N' en la columna 'tipo_identificacion'.
   2. Para placas repetidas:
      - Elimina el registro donde 'identificacion' comienza con el primer valor de identificación.
      - Asigna 'C' al segundo valor de 'tipo_identificacion' cuando los valores de 'identificacion' son diferentes.
   :param dataframe: DataFrame con las columnas 'placa', 'identificacion', 'tipo_identificacion'.
   :return: DataFrame procesado.
    """
   # Limpiar la columna 'identificacion' eliminando '.0' si existe
    dataframe['identificacion'] = dataframe['identificacion'].apply(lambda x: str(x).split('.')[0])
   # Filtrar las filas con 'N' en 'tipo_identificacion'
    df_n = dataframe[dataframe['tipo_identificacion'] == 'N']
    resultados = []
   # Procesar placas duplicadas
    for placa, grupo in df_n.groupby('placa'):
        if len(grupo) > 1:  # Procesar solo placas repetidas
            identificaciones = grupo['identificacion'].tolist()
            tipo_identificaciones = grupo['tipo_identificacion'].tolist()
            for i in range(1, len(identificaciones)):
               # Caso 1: Identificaciones que comienzan con el primer valor (duplicados)
                if identificaciones[i].startswith(identificaciones[0]):
                    tipo_identificaciones[i] = None  # Marca para eliminar
               # Caso 2: Identificaciones diferentes
                elif identificaciones[i] != identificaciones[0]:
                    tipo_identificaciones[i] = 'C'
           # Actualizar el grupo con los cambios
            grupo['tipo_identificacion'] = tipo_identificaciones
            grupo = grupo.dropna(subset=['tipo_identificacion'])  # Elimina filas marcadas
            resultados.append(grupo)
        else:
            resultados.append(grupo)
   # Combinar los resultados procesados
    df_n_procesado = pd.concat(resultados)
   # Combinar los valores procesados con los no filtrados
    df_resultado = pd.concat([df_n_procesado, dataframe[dataframe['tipo_identificacion'] != 'N']], ignore_index=True)
    return df_resultado

# Cargar el archivo generado anteriormente
existing_dataframe = pd.read_excel("pruebarunt.xlsx")
# Crear una lista vacía para almacenar las filas del nuevo dataframe
new_data = []
# Diccionario para convertir los tipos de identificación a letras
tipo_identificacion_map = {
   'NIT': 'N',
   'Cédula Ciudadanía': 'C',
    'Cédula ciudadanía': 'C',
   'Cédula Extranjería': 'E',
    'Cédula extranjería': 'E',
   'Tarjeta Identidad': 'T',
   'Pasaporte': 'P'
}
# Recorrer el dataframe existente
for index, row in existing_dataframe.iterrows():
    tipo_identificacion = tipo_identificacion_map.get(row['tipo_iden_aseg'], row['tipo_iden_aseg'])  # Mapear tipo de identificación
    if row['igual'] == 1:
       # Agregar una única fila al nuevo dataframe
        new_data.append({
            'placa': row['placa_vehiculo'],
            'identificacion': str(row['nro_iden_aseg']).split('.')[0],
            'tipo_identificacion': tipo_identificacion
        })
    else:
       # Agregar dos filas al nuevo dataframe para los casos sin coincidencia
        new_data.append({
            'placa': row['placa_vehiculo'],
            'identificacion': str(row['nro_iden_aseg']).split('.')[0],
            'tipo_identificacion': tipo_identificacion
        })
        new_data.append({
            'placa': row['placa_vehiculo'],
            'identificacion': str(row['nro_iden_conduc']).split('.')[0],
            'tipo_identificacion': tipo_identificacion
        })
# Crear el nuevo dataframe a partir de la lista de datos
new_dataframe = pd.DataFrame(new_data)
# Aplicar la función de procesamiento a las placas
processed_dataframe = procesar_placas(new_dataframe)
# Guardar el archivo procesado
# processed_dataframe.to_excel("muestra_Liberty_INTEMPO_30_DIC_2024_procesado.xlsx", index=False)
# # Ver el resultado en consola (opcional)
# print(processed_dataframe)

In [58]:
# Generar el nombre del archivo basado en la fecha actual
meses = ['ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC']
hoy = datetime.now()
nombre_archivo = f"muestra_Liberty_INTEMPO_{hoy.strftime('%d')}_{meses[hoy.month - 1]}_{hoy.strftime('%Y')}.xlsx"
# Guardar el archivo procesado
processed_dataframe.to_excel(nombre_archivo, index=False)
print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: muestra_Liberty_INTEMPO_13_ENE_2025.xlsx
